In [1]:
import time
import sqlite3
import pandas as pd
from futu import *


In [ ]:
# ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
# code_list = ['SZ.300212']
# print(ctx.get_owner_plate(code_list))
# quote_ctx.close()

In [ ]:
plate_list_df = ctx.get_plate_list(Market.SH, Plate.CONCEPT)[1]
plate_list_df.head()

In [ ]:
# concept_dashboard_code = pd.DataFrame()

# bk0300_df = ctx.get_plate_stock('SH.BK0300')[1]
# # concept_dashboard_code['0300'] = bk0300_df['code']
# concept_dashboard_code = pd.concat([concept_dashboard_code, bk0300_df['code']], axis=1)

# # bk0301_df = ctx.get_plate_stock('SH.BK0301')[1]
# # # concept_dashboard_code['0301'] = bk0301_df['code']
# concept_dashboard_code = pd.concat([concept_dashboard_code, bk0301_df['code']], axis=1)

# # bk0305_df = ctx.get_plate_stock('SH.BK0305')[1]
# # # concept_dashboard_code['0305'] = bk0305_df['code']
# concept_dashboard_code = pd.concat([concept_dashboard_code, bk0305_df['code']], axis=1)

# # concept_dashboard_code
    

In [ ]:
def update_concept_db(dpath):
    concept_list_df = ctx.get_plate_list(Market.SH, Plate.CONCEPT)[1]
    
    concept_dashboard_code = pd.DataFrame()
    concept_dashboard_name = pd.DataFrame()
    counter = 0
    for idx, row in concept_list_df.iterrows():
        print(str(idx) + '  ' + str(row['plate_name']))
        plate_df = ctx.get_plate_stock(row['code'])[1] # 30 秒内请求最多 10 次
        
        concept_dashboard_code = pd.concat([concept_dashboard_code, plate_df['code']], axis=1)
        concept_dashboard_name = pd.concat([concept_dashboard_name, plate_df['stock_name']], axis=1)
        concept_dashboard_code.rename(columns={'code':row['plate_name']}, inplace=True)
        concept_dashboard_name.rename(columns={'stock_name':row['plate_name']}, inplace=True)
        
        counter = counter+1
        if (counter > 9):
            counter = 0
            time.sleep(30)
    
    sql_ctx = sqlite3.connect(dpath)
    concept_list_df.to_sql('concept_list', sql_ctx, if_exists='replace')
    concept_dashboard_code.to_sql('concept_dashboard_code', sql_ctx, if_exists='replace')
    concept_dashboard_name.to_sql('concept_dashboard_name', sql_ctx, if_exists='replace')

In [ ]:
# update_concept_db('/Users/linhua/PycharmProjects/Fupan/database/futu.db')
# ctx.close()


In [ ]:
ctx.close()

In [2]:
sql_cmd = "select * from concept_dashboard_name"
context = sqlite3.connect('/Users/linhua/PycharmProjects/Fupan/database/futu.db')
df = pd.read_sql(sql_cmd, context, index_col='index')

In [3]:
df

,中韩自贸区,汽车电子概念,高送转,上海自贸区,电力物联网,养鸡,医疗器械,燃料电池,乳业,黄金,...,光刻机,上证50,深圳特区,分拆上市预期,数字货币,VPN,医疗美容,农药兽药,中药,双黄连概念
index,,,,,,,,,,,,,,,,,,,,,
0,青岛双星,富奥股份,泰禾集团,悦心健康,许继电气,康达尔,海王生物,华锦股份,新希望,中国宝安,...,智光电气,浦发银行,平安银行,中国宝安,中国长城,*ST高升,苏宁环球,长春高新,宜华健康,葵花药业
1,冰轮环境,深圳华强,华兰生物,怡亚通,东方电子,新希望,英特集团,潍柴动力,皇氏集团,中金岭南,...,永太科技,民生银行,万科A,深康佳A,智度股份,南洋股份,景峰医药,泰合健康,ST地矿,太龙药业
2,中广核技,风华高科,双鹭药业,海得控制,思源电气,民和股份,海南海药,东旭光电,大康农业,ST地矿,...,胜利精密,宝钢股份,国农科技,铜陵有色,恒宝股份,卫士通,华东医药,海翔药业,东阿阿胶,天士力
3,青岛金王,中鼎股份,中钢天源,*ST飞马,远光软件,圣农发展,京东方A,穗恒运A,洽洽食品,中润资源,...,西陇科学,中国石化,世纪星源,长春高新,广电运通,启明星辰,双鹭药业,天康生物,丽珠集团,哈药股份
4,朗姿股份,航天科技,金智科技,滨江集团,金智科技,华英农业,重药控股,江铃汽车,贝因美,金圆股份,...,光华科技,南方航空,深振业A,盈峰环境,亚联发展,二六三,朗姿股份,金河生物,云南白药,ST辅仁
5,日照港,华工科技,三钢闽光,中远海科,科陆电子,益生股份,欣龙控股,苏常柴A,麦趣尔,金洲慈航,...,宝通科技,中信证券,全新好,华邦健康,众应互联,南兴股份,哈三联,绿康生化,启迪古汉,人民同泰
6,锦州港,宁波华翔,东港股份,摩恩电气,智光电气,仙坛股份,京新药业,*ST大洲,燕塘乳业,盛达资源,...,金龙机电,三一重工,中国宝安,东港股份,金固股份,朗源股份,冠昊生物,瑞普生物,吉林敖东,珍宝岛
7,时代万恒,得润电子,湘潭电化,华测检测,威创股份,华统股份,科华生物,英力特,科迪乳业,银泰黄金,...,上海新阳,招商银行,*ST美丽,西部材料,海联金汇,蓝盾股份,常山药业,东湖高新,仁和药业,None
8,营口港,顺络电子,合肥城建,新宁物流,积成电子,温氏股份,达安基因,茂化实华,庄园牧场,ST冠福,...,蓝英装备,保利地产,深物业A,广电运通,奥马电器,任子行,鲁商发展,中牧股份,通化金马,None
